In [1]:
shuffle_data_file = "../temp/out/dev_matched_shuffled.csv"
shuffle_data_pred_file = "../temp/out/dev_matched_shuffle_revese_finetune_predict.txt"
shuffle_data_pred_file = "../temp/out/dev_matched_shuffled_bert_base_finetune_predict.txt"



data_file = "../temp/out/dev_matched.csv"
data_pred_file = "../temp/out/dev_matched_reverse_finetune_predict.txt"
data_pred_file = "../temp/out/dev_matched_bert_base_finetune_predict.txt"


In [2]:
import pandas as pd
pd.set_option('max_colwidth', 0)

In [3]:
def load_df(raw_file, pred_file):

    df = pd.read_csv(raw_file, quotechar='"')
    df["prediction"] = pd.read_csv(pred_file, delimiter="\t")["prediction"]
    df = df.drop(columns = ["sentence1_binary_parse","sentence2_binary_parse", "sentence1_parse", "sentence2_parse"])
    return df

In [ ]:
from sklearn import tree
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from nltk import ngrams

def overlap_words(df, cola, colb):
    df["overlap"] = df.apply(lambda x:len( set(x[cola].split(" ")).intersection(set(x[colb].split(" "))))/len(set(x[cola].split(" ")).union(set(x[colb].split(" ")))), axis=1)
   # df["overlap"] = df.apply(lambda x:len( set(x[cola].split(" "))-(set(x[colb].split(" "))))/len(set(x[cola].split(" "))), axis=1)
   
    return df

def overlap_ngrams(df,cola, colb, ngram=1):
    ngram_counts = Counter(ngrams(x, 2))
    df[f"ngramcount_{ngram}"] = df.apply(lambda x: Counter(ngrams("{} {}".format( x[cola], x[col]), 2)).most_common(0)[0][1])
    
    
    return df

def has_not(df,  cola, colb):
    df["has_not"] = df.apply(lambda x: ("not" in x[colb] and "not" not in x[cola]) or ("not" in x[cola] and "not" not in x[colb])  , axis=1)
    return df

def length_diff(df,  cola, colb ):
    df["length_diff"] = df.apply(lambda x:  len(x[cola].split(" "))-len(x[colb].split(" ")), axis=1)
    return df

def length(df,  col, result_col ):
    df[result_col] = df.apply(lambda x:  len(x[col].split(" ")), axis=1)
    return df

def vectoriser(df, col):
    vec = CountVectorizer(stop_words=None, 
                                    ngram_range=(2,3), min_df=3, max_features=500)
    vec.fit(df[col])
    return vec.transform(df[col])

def train_fit(df,s1,s2, y):
    model = tree.DecisionTreeClassifier(max_depth=5)
    overlap_feature = overlap_words(df, s1,s2)["overlap"].values
   # has_not_feature = has_not(df, s1, s2)["has_not"].values
    length_diff_feature = length_diff(df,s1,s2)["length_diff"].values
    lengtha_feature = length(df,s2,"length_a")["length_a"].values
    ngram=3
    overlap_ngrams_feature = = overlap_ngrams(df,s1,s2,ngram=ngram)[f"ngramcount_{ngram}"].values
   # lengthb_feature = length(df,s2,"length_b")["length_b"].values
    rule_features = np.vstack((  overlap_feature, length_diff_feature,lengtha_feature, overlap_ngrams_feature)).T
    vector = vectoriser(df, s1).toarray()
    #features = np.concatenate((rule_features,vector ), axis=1)
    features = rule_features
    model.fit(features, df[y])
    df["tree_prediction"]= model.predict(features)
    return df


In [161]:
df_shuffle =  load_df(shuffle_data_file, shuffle_data_pred_file)
df_normal =  load_df(data_file, data_pred_file)


In [ ]:
df_normal = train_fit(df_normal, "sentence1", "sentence2", "label")
df_normal.query("tree_prediction == label").shape[0]/df_normal.shape[0], df_normal.query("tree_prediction == prediction").shape[0]-df_normal.shape[0]

In [295]:
df_normal["label"].value_counts()

000_entailment       3441
002_contradiction    3179
001_neutral          3094
Name: label, dtype: int64

In [350]:
df_normal["tree_prediction"].value_counts()

000_entailment       4170
001_neutral          2913
002_contradiction    2631
Name: tree_prediction, dtype: int64

In [297]:
df_normal["prediction"].value_counts()

000_entailment       3383
002_contradiction    3232
001_neutral          3099
Name: prediction, dtype: int64

In [172]:
df_normal.query("tree_prediction == prediction")["label"].value_counts()

000_entailment       1935
001_neutral          1432
002_contradiction    1291
Name: label, dtype: int64

In [186]:
df_normal.query("tree_prediction != prediction").sample(n=10)

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction,overlap,length_diff,length_b,tree_prediction,has_not
808,808,36262,36262e,telephone,other side that's a good idea,There is more than one side.,entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment,0.000000,0,6,001_neutral,False
4501,4551,123103,123103n,fiction,Vrenna looked it and smiled.,Vrenna was happy it was destroyed,neutral,neutral,contradiction,neutral,neutral,001_neutral,001_neutral,0.250000,-1,6,000_entailment,False
8568,8669,50186,50186e,government,Neither does it include the mail sent in response to advertising.,It does not include the mail sent in response to advertising.,entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment,0.692308,0,11,002_contradiction,True
8236,8337,71327,71327e,fiction,Bien! he said at last.,He finally blurted something out.,entailment,neutral,entailment,entailment,entailment,000_entailment,000_entailment,0.000000,1,5,002_contradiction,False
9345,9446,58872,58872c,fiction,"When Mr. Hastings and Mr. Lawrence came in yesterday evening, they found your mistress busy writing letters.",Last night no one saw your mistress.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.043478,11,7,001_neutral,False
7525,7626,99279,99279c,government,Title IV of the Clean Air Act (relating to acid deposition control),The Clean Air Act only has a single title.,contradiction,neutral,contradiction,neutral,contradiction,002_contradiction,002_contradiction,0.166667,3,9,000_entailment,False
2644,2694,80803,80803c,telephone,well uh what do you think about taxes do you think we're paying too much,We pay just the right amount of taxes.,contradiction,neutral,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.000000,7,8,001_neutral,False
8889,8990,65566,65566n,slate,"And if, as ultimately happened, no settlement resulted, we could shrug our shoulders, say, 'Hey, we tried,' and act like unsuccessful brokers to an honorable peace.",Neither side is actually interested in a settlement at this time.,neutral,contradiction,neutral,neutral,neutral,001_neutral,002_contradiction,0.028571,15,11,001_neutral,False
1391,1391,141293,141293n,telephone,oh wow no i just started about well five years ago i think,The first year was rough.,neutral,neutral,contradiction,neutral,neutral,001_neutral,001_neutral,0.000000,8,5,002_contradiction,False
5945,6010,18874,18874c,government,"You wonder what youre going to be when you grow up, lawyer Smith said.",You have no dreams for the future.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.047619,8,7,001_neutral,False


In [61]:
df_normal["label"].value_counts()

000_entailment       3441
002_contradiction    3179
001_neutral          3094
Name: label, dtype: int64

In [60]:
df_normal.query("tree_prediction == label")["label"].value_counts()

000_entailment       2266
001_neutral          974 
002_contradiction    945 
Name: label, dtype: int64

In [228]:
df_normal.head(n=10)

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction,overlap,length_diff,length_b,tree_prediction,has_not
0,0,63735,63735n,slate,The new rights are nice enough,Everyone really likes the newest benefits,neutral,entailment,neutral,neutral,neutral,001_neutral,001_neutral,0.000000,-1,7,002_contradiction,False
1,1,91383,91383c,government,This site includes a list of all award winners and a searchable database of Government Executive articles.,The Government Executive articles housed on the website are not able to be searched.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.074074,3,14,002_contradiction,True
2,2,755,755e,telephone,uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him,"I like him for the most part, but would still enjoy seeing someone beat him.",entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment,0.161290,13,15,000_entailment,False
3,3,78013,78013c,telephone,yeah i i think my favorite restaurant is always been the one closest you know the closest as long as it's it meets the minimum criteria you know of good food,My favorite restaurants are always at least a hundred miles away from my house.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.111111,17,15,001_neutral,False
4,4,96377,96377c,telephone,i don't know um do you do a lot of camping,I know exactly.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.083333,8,3,002_contradiction,False
5,5,139749,139749c,telephone,well that would be a help i wish they would do that here we have got so little landfill space left that we're going to run out before the end of this decade and it's really going to be,We have plenty of space in the landfill.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.108108,31,8,002_contradiction,False
6,6,101415,101415c,telephone,yeah i know and i did that all through college and it worked too,I did that all through college but it never worked,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.437500,3,11,000_entailment,False
7,7,93958,93958n,travel,"Calcutta seems to be the only other production center having any pretensions to artistic creativity at all, but ironically you're actually more likely to see the works of Satyajit Ray or Mrinal Sen shown in Europe or North America than in India itself.",Most of Mrinal Sen's work can be found in European collections.,neutral,neutral,entailment,neutral,neutral,001_neutral,001_neutral,0.088889,32,11,001_neutral,False
8,8,12567,12567c,slate,"If that investor were willing to pay extra for the security of limited downside, she could buy put options with a strike price of $98, which would lock in her profit on the shares at $18, less whatever the options cost.",THe strike price could be $8.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction,0.075000,35,6,002_contradiction,False
9,9,117487,117487n,slate,"3) Dare you rise to the occasion, like Raskolnikov, and reject the petty rules that govern lesser men?",Would you rise up and defeaat all evil lords in the town?,neutral,neutral,neutral,neutral,neutral,001_neutral,001_neutral,0.153846,7,12,000_entailment,False


In [32]:
df_normal = df_normal.pipe(overlap_words, "sentence1", "sentence2")

In [33]:
df_normal[["label", "prediction", "overlap" ]]

,label,prediction,overlap
0,001_neutral,001_neutral,0.000000
1,002_contradiction,002_contradiction,0.074074
2,000_entailment,000_entailment,0.161290
3,002_contradiction,002_contradiction,0.111111
4,002_contradiction,002_contradiction,0.083333
...,...,...,...
9709,001_neutral,001_neutral,0.111111
9710,002_contradiction,002_contradiction,0.368421
9711,000_entailment,000_entailment,0.115385
9712,000_entailment,000_entailment,0.142857


In [5]:
df_normal.shape

(9714, 13)

In [27]:
df_normal.query("index == 9769")

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction
9668,9769,96377,96377e,telephone,i don't know um do you do a lot of camping,Do you camp alot?,entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment


In [ ]:
df_normal.head()

In [7]:
df_normal.head()

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction
0,0,63735,63735n,slate,The new rights are nice enough,Everyone really likes the newest benefits,neutral,entailment,neutral,neutral,neutral,001_neutral,001_neutral
1,1,91383,91383c,government,This site includes a list of all award winners and a searchable database of Government Executive articles.,The Government Executive articles housed on the website are not able to be searched.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction
2,2,755,755e,telephone,uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him,"I like him for the most part, but would still enjoy seeing someone beat him.",entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment
3,3,78013,78013c,telephone,yeah i i think my favorite restaurant is always been the one closest you know the closest as long as it's it meets the minimum criteria you know of good food,My favorite restaurants are always at least a hundred miles away from my house.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction
4,4,96377,96377c,telephone,i don't know um do you do a lot of camping,I know exactly.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction


In [8]:
df_normal.shape

(9714, 13)

In [9]:
df_shuffle[["sentence1", "sentence2"]].sample(n=3)

,sentence1,sentence2
3211,"of street the university life, vigorous aerosece local culture enthusiasm and to with Ceter has city national a bright an infectious and this it. breezy industry,","uninspired staid, The stuffy, city a has feeling it. to"
3166,welcome and restaurants Many cafes children.,are restaurants along the cafes. take to with you many and to great Children
2029,"sooner Tuppence. The better."" we strike to turned He the",He the to dropping about talked Tuppence bomb immediately.


In [10]:
df_shuffle["is_normal_correct"] = df_shuffle["index"].isin( df_normal.query("label == prediction")["index"])

In [22]:
 df_normal.query("index == 11")

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction
11,11,119637,119637c,travel,The bhakti movement of the Tamils brought a new warmth to the hitherto rigid Brahmanic ritual of Hinduism.,The Tamils' bhakti movement froze the previously warm ritual of Hinduism.,contradiction,contradiction,contradiction,contradiction,entailment,002_contradiction,002_contradiction


In [11]:
df_shuffle.query("label != prediction and label != '001_neutral' and is_normal_correct ==True ").shape

(940, 14)

In [26]:
df_shuffle.query("label != prediction and label != '001_neutral' and prediction != '001_neutral' and is_normal_correct ==True ")\
    [["index", "sentence1", "sentence2", "label", "prediction"]]

,index,sentence1,sentence2,label,prediction
11,11,Hinduism. The rigid new bhakti hitherto ritual Tamils the to of warmth of brought a the Brahmanic movement,froze bhakti The ritual the Tamils' warm of Hinduism. movement previously,002_contradiction,000_entailment
13,13,"and your friends You are not welcome Severn. said here,",Severn said were not people there. welcome the,000_entailment,002_contradiction
45,45,file their uh on from two send uh had guys to were but road them final actually in they when the home exams from uh their thousand these miles and,exams their filed home. men midterm These from,002_contradiction,000_entailment
58,58,"a its heightened possibilities. a has and She turns, its has to tried life hollow get exchanged comprehend all life, for",hollow a life. live chose to has She,002_contradiction,000_entailment
72,72,"those piles Alliance. of no Cook's There as a worthless that just shares back, of might stock are shares someday as of American Business come options",Cook's to Business shares Alliance caused back. of stock come American,002_contradiction,000_entailment
...,...,...,...,...,...
9631,9732,"includes There analysis, tradition no opera who of among do ballet, critics, theater, writerly criticism even is august or criticism clothes serious have that of and costume heritage. an",criticism is Clothes serious.,002_contradiction,000_entailment
9668,9769,i camping a of know you do don't lot do um,you Do alot? camp,000_entailment,002_contradiction
9673,9774,"as blamed until paying is to globalization's official excesses we Rightly its America be or prime head and start mover seen and them. will for wrongly, cheerleader attention",role globalization in important with it movement America's not. we agree is the whether or,000_entailment,002_contradiction
9676,9777,hillsides; and many into selling into homes some shops the art converted been are have collectibles. There galleries built,hillside in and selling the have collectibles. shops homes of been converted art galleries the All into,002_contradiction,000_entailment


In [21]:
df_shuffle.query("label == prediction and label != '001_neutral' and is_normal_correct ==True ")\
    [["sentence1", "sentence2", "label", "prediction"]]

,sentence1,sentence2,label,prediction
1,a This and database winners articles. a all award of searchable list includes Executive of site Government,to searched. Government on be are Executive the not website The housed able articles,002_contradiction,002_contradiction
2,somebody about at see i know the to uh times him sometimes beat him have i don't i uh emotions like i but mixed i same love him,"most still him. like but him for would seeing enjoy I part, beat the someone",000_entailment,000_entailment
3,as criteria it's know the closest my one favorite food you always think as restaurant yeah good been minimum is long it i the of the i meets closest you know,hundred away My my favorite at a always miles from restaurants least house. are,002_contradiction,002_contradiction
4,know do of um do a camping lot don't i you,know I exactly.,002_contradiction,002_contradiction
5,out that got would we're before to well we that would of wish here they i it's little space left that end landfill so be have really the be this help and going do decade a going run to,We plenty have of the in space landfill.,002_contradiction,002_contradiction
...,...,...,...,...
9708,"Second, rate not If entirely. of the only we'd any goal it reducing were, sex worth the social in HIV is event transmission outlaw",HIV problems. No one about social or other cares any,002_contradiction,002_contradiction
9710,in by malfunction space. were pagers the United knocked out of percent satellite a States Eighty in,by Pagers States the in United the satellite were unaffected malfunction.,002_contradiction,002_contradiction
9711,"the videotapes, the and entities complying in issue to conduct aid with hold FDA guidance will manuals rule. and Finally, workshops, teleconferences small",is set to workshops. FDA The conduct,000_entailment,000_entailment
9712,"Soleil's that of 1.5 in water. an The million troupe, from aquatic latest liters) million international utilizes gallons dazzles acclaimed O the du environment (6.8 Cirque",an Cirque international troupe. is du Soleil,000_entailment,000_entailment


In [14]:
df_shuffle.query("label == prediction and label != '001_neutral' and is_normal_correct ==True ").shape, df_shuffle.query("label != '001_neutral' and is_normal_correct ==True ").shape

((4754, 14), (5694, 14))

In [15]:
df_shuffle.query("label != '001_neutral'")["is_normal_correct"].value_counts()

True     5694
False    926 
Name: is_normal_correct, dtype: int64

In [16]:
df_shuffle["is_normal_correct"].value_counts()

True     8165
False    1549
Name: is_normal_correct, dtype: int64

In [17]:
(df_normal["label"]==df_normal["prediction"]).value_counts()

True     8165
False    1549
dtype: int64